# Title
[]()

In [6]:
import os

# documents
from langchain.document_loaders import DirectoryLoader
# from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool

# Creating the Agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory import ConversationBufferMemory

# Create the chain
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from langchain.prompts import PromptTemplate


# Initialize Dictionaries

In [2]:
tool_dict = dict()
embeddings_dict = dict()
db_dict = dict()
retriever_dict = dict()
vector_dict = dict()
description_dict = dict()

In [12]:
answer_dict=dict()
conversation_dict = dict()

In [41]:
doc_dict = dict()
tools_list = create_tools_list(retriever_dict, description_dict)

# iteration 1

In [3]:

def create_documents(directory='../data/', glob='**/*.txt', show_progress=True):
    loader = DirectoryLoader(directory, glob=glob, show_progress=show_progress)
    documents = loader.load()
    print(f'Number of files: {len(documents)}')
    return documents

documents = create_documents()
documents

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:06<00:00,  3.14s/it]

Number of files: 2


[Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/apartments\n\n\n\ncondos\n\n\n\nand\n\n\n\ntownhomes.aspx\n\nApartments, condos, and townhomes\n\nWe do not provide waste collection services to most multi-unit buildings.\n\nHowever, we do provide information and resources to building owners and managers to manage waste at apartments, condos, and townhomes.\n\nWaste Management Canada collects your recycling on behalf of Recycle BC External website, opens in new tab Phone Waste Management Canada at 604-282-7961 for questions and issues about your recycling service.', metadata={'source': '..\\data\\Vancouver apartments condos townhomes.txt'}),
 Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/single\n\n\n\nfamily\n\n\n\nhomes\n\n\n\nand\n\n\n\nduplexes.aspx\n\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes an

In [13]:
# https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.CharacterTextSplitter.html?highlight=charactertextsplitter#langchain.text_splitter.CharacterTextSplitter.split_documents
text_splitter = CharacterTextSplitter(chunk_size=30, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
texts

Created a chunk of size 33, which is longer than the specified 30
Created a chunk of size 73, which is longer than the specified 30
Created a chunk of size 134, which is longer than the specified 30
Created a chunk of size 130, which is longer than the specified 30
Created a chunk of size 67, which is longer than the specified 30
Created a chunk of size 234, which is longer than the specified 30


[Document(page_content='City of Vancouver', metadata={'source': '..\\data\\Vancouver apartments condos townhomes.txt'}),
 Document(page_content='https://vancouver.ca/home', metadata={'source': '..\\data\\Vancouver apartments condos townhomes.txt'}),
 Document(page_content='property', metadata={'source': '..\\data\\Vancouver apartments condos townhomes.txt'}),
 Document(page_content='development/apartments\n\ncondos', metadata={'source': '..\\data\\Vancouver apartments condos townhomes.txt'}),
 Document(page_content='and\n\ntownhomes.aspx', metadata={'source': '..\\data\\Vancouver apartments condos townhomes.txt'}),
 Document(page_content='Apartments, condos, and townhomes', metadata={'source': '..\\data\\Vancouver apartments condos townhomes.txt'}),
 Document(page_content='We do not provide waste collection services to most multi-unit buildings.', metadata={'source': '..\\data\\Vancouver apartments condos townhomes.txt'}),
 Document(page_content='However, we do provide information and 

In [15]:
print(type(texts[0]))
print(texts[0].page_content)

<class 'langchain.schema.document.Document'>
City of Vancouver


In [4]:
def create_retriever(documents, site_key, vector_dict=vector_dict, text_splitter=None):
    """
    Parameters:
        - text_splitter (optional): a text splitter object. If None, the documents are not split. 
    """
    embeddings_dict[site_key] = OpenAIEmbeddings()
    if text_splitter is None: # object type is the same (class 'langchain.schema.document.Document') whether or not the documents are split
        texts = documents
    else:
        texts = text_splitter.split_documents(documents)

    vector_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
    retriever_dict[site_key] = vector_dict[site_key].as_retriever()
    return retriever_dict

retriever_dict = create_retriever(documents, 'CoV')
retriever_dict


{'CoV': VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x00000226CA1DC5D0>, search_type='similarity', search_kwargs={})}

In [18]:
type(retriever_dict['CoV'])

langchain.vectorstores.base.VectorStoreRetriever

In [24]:
retriever_dict.items()

dict_items([('CoV', VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x0000029FE0CCAF50>, search_type='similarity', search_kwargs={}))])

In [32]:
retriever_dict['CoV']

VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x0000029FE0CCAF50>, search_type='similarity', search_kwargs={})

## Create tools list

In [5]:

def create_tools_list(retriever_dict, description_dict):
    """
    https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html?highlight=create_retriever_tool#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool
    """
    tools_list = []
    for site_key, retriever in retriever_dict.items():
        tool_name = f'search_{site_key}'
        tool = create_retriever_tool(retriever_dict[site_key], tool_name, description_dict[site_key])
        tools_list.append(tool)
    return tools_list

description_dict['CoV'] = 'Searches and returns documents regarding waste and recycling in the City of Vancouver.'

tools_list = create_tools_list(retriever_dict, description_dict)
tools_list



[Tool(name='search_CoV', description='Searches and returns documents regarding waste and recycling in the City of Vancouver.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x00000226CA1DC5D0>, search_type='similarity', search_kwargs={})>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x00000226CA1DC5D0>, search_type='similarity', search_kwargs={})>)]

## Create chatbot
https://api.python.langchain.com/en/latest/chat_models/langchain.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#langchain.chat_models.openai.ChatOpenAI

Need to customize prompt template

In [14]:


def create_chatbot(tools_list=tools_list, verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools_list, verbose)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    template = (
        "Combine the chat history and follow up question into "
        "a standalone question. Chat History: {chat_history}"
        "Follow up question: {question}"
    )
    # prompt = PromptTemplate.from_template(template)
    # question_generator_chain = LLMChain(llm=llm, prompt=prompt)
    # chain = ConversationalRetrievalChain(question_generator=question_generator_chain)
    # qa = chain.from_llm(
    #     llm, vector_dict[site_key].as_retriever(), memory=memory
    #     )
    # result = qa({"question": user_input})
    chain = ConversationalRetrievalChain()
    template = """You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
    To make your answer more concise, you ask follow up questions if needed so you can provide the most relevant answer.
    Given the following conversation and a follow up question, rephrase the follow up question 
    to be a standalone question, in its original language.\n\n
    Chat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:
    """
    prompt = PromptTemplate(
        input_variables=['chat_history', 'question'], 
        output_parser=None, partial_variables={}, 
        template=template, template_format='f-string', validate_template=True)
    chat = chain.from_llm(
        llm, vector_dict[site_key].as_retriever(), memory=memory,
        condense_question_prompt=prompt
        )

    return chat

def chat_with_chatbot(user_input, chat, verbose=True):
    result = qa({"question": user_input})
    
    return result

conversation_id = 1
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[input_id] = chat_with_chatbot(query, chat_dict[conversation_id])

ValidationError: 3 validation errors for ConversationalRetrievalChain
combine_docs_chain
  field required (type=value_error.missing)
question_generator
  field required (type=value_error.missing)
retriever
  field required (type=value_error.missing)

### iteration 2

In [17]:


def create_chatbot(tools_list=tools_list, verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools_list, verbose)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    template = (
        "Combine the chat history and follow up question into "
        "a standalone question. Chat History: {chat_history}"
        "Follow up question: {question}"
    )
    template = """You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
    To make your answer more concise, you ask follow up questions if needed so you can provide the most relevant answer.
    Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
    Here is your chat history with the resident: \n\n{chat_history}\n\n
    Respond to the resident's query, which are delimited by triple backticks: ```{question}```
    """
    prompt = PromptTemplate(
        input_variables=['chat_history', 'question'], 
        output_parser=None, partial_variables={}, 
        template=template, template_format='f-string', validate_template=True)
    chat = ConversationalRetrievalChain.from_llm(
        llm, vector_dict[site_key].as_retriever(), memory=memory,
        condense_question_prompt=prompt
        )

    return chat

def chat_with_chatbot(user_input, chat, verbose=True):
    result = qa({"question": user_input})
    
    return result

conversation_id = 1
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[input_id] = chat_with_chatbot(query, chat_dict[conversation_id])

NameError: name 'site_key' is not defined

### iteration 3

In [21]:


def create_chatbot(tools_list=tools_list, vector_store=vector_dict['CoV'], verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools_list, verbose)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    template = (
        "Combine the chat history and follow up question into "
        "a standalone question. Chat History: {chat_history}"
        "Follow up question: {question}"
    )
    template = """You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
    To make your answer more concise, you ask follow up questions if needed so you can provide the most relevant answer.
    Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
    Here is your chat history with the resident: \n\n{chat_history}\n\n
    Respond to the resident's query, which are delimited by triple backticks: ```{question}```
    """
    prompt = PromptTemplate(
        input_variables=['chat_history', 'question'], 
        output_parser=None, partial_variables={}, 
        template=template, template_format='f-string', validate_template=True)
    chat = ConversationalRetrievalChain.from_llm(
        llm, vector_store.as_retriever(), memory=memory,
        condense_question_prompt=prompt
        )

    return chat

def chat_with_chatbot(user_input, chat, verbose=True):
    result = chat({"question": user_input})
    
    return result

conversation_id = 1
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])

In [22]:
answer_dict[input_id]

{'question': 'Where do I recycle coffee cups in Vancouver?',
 'chat_history': [HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False),
  AIMessage(content='For single-family homes and duplexes in Vancouver, you can recycle coffee cups in your blue bin provided by GFL Environmental. You can contact GFL Environmental at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service, including coffee cup recycling.\n\nFor apartments, condos, and townhomes, waste collection services are not provided by the City of Vancouver. Waste Management Canada collects recycling on behalf of Recycle BC for these types of buildings. You can contact Waste Management Canada at 604-282-7961 for questions and issues about recycling, including coffee cup recycling.', additional_kwargs={}, example=False)],
 'answer': 'For single-family homes and duplexes in Vancouver, you can recycle coffee cups in your blue bin provi

In [26]:
print(len(answer_dict[input_id]['answer']))
print(answer_dict[input_id]['answer'])

636
For single-family homes and duplexes in Vancouver, you can recycle coffee cups in your blue bin provided by GFL Environmental. You can contact GFL Environmental at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service, including coffee cup recycling.

For apartments, condos, and townhomes, waste collection services are not provided by the City of Vancouver. Waste Management Canada collects recycling on behalf of Recycle BC for these types of buildings. You can contact Waste Management Canada at 604-282-7961 for questions and issues about recycling, including coffee cup recycling.


In [27]:
answer_dict[input_id]['chat_history']

[HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False),
 AIMessage(content='For single-family homes and duplexes in Vancouver, you can recycle coffee cups in your blue bin provided by GFL Environmental. You can contact GFL Environmental at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service, including coffee cup recycling.\n\nFor apartments, condos, and townhomes, waste collection services are not provided by the City of Vancouver. Waste Management Canada collects recycling on behalf of Recycle BC for these types of buildings. You can contact Waste Management Canada at 604-282-7961 for questions and issues about recycling, including coffee cup recycling.', additional_kwargs={}, example=False)]

In [28]:
input_id = 2

query = "I live in an apartment. Who should I talk to so I can get a blue bin?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])

In [29]:
answer_dict[input_id]['chat_history']

[HumanMessage(content='I live in an apartment. Who should I talk to so I can get a blue bin?', additional_kwargs={}, example=False),
 AIMessage(content='For apartments, condos, and townhomes, the City of Vancouver does not provide waste collection services. However, you can contact Waste Management Canada at 604-282-7961 for questions and issues about your recycling service. They should be able to assist you with obtaining a blue bin for your apartment.', additional_kwargs={}, example=False)]

In [30]:
print(answer_dict[input_id]['answer'])

For apartments, condos, and townhomes, the City of Vancouver does not provide waste collection services. However, you can contact Waste Management Canada at 604-282-7961 for questions and issues about your recycling service. They should be able to assist you with obtaining a blue bin for your apartment.


# Iteration 2 Update it to work with CSV

In [60]:

def create_documents(directory='../data/', glob='**/[!.]*', show_progress=True, loader_cls=CSVLoader):
    loader = DirectoryLoader(
        directory, glob=glob, show_progress=show_progress,
        loader_cls=loader_cls)

    documents = loader.load()
    print(f'Number of files: {len(documents)}')
    return documents

doc_id = 1
doc_dict[doc_id] = create_documents(glob='**/*.csv')
doc_dict[doc_id]

100%|██████████| 1/1 [00:00<00:00, 64.14it/s]

Number of files: 158


[Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Paper\nItem: Newspapers, inserts and flyers', metadata={'source': '..\\data\\recyclebc.csv', 'row': 0}),
 Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Paper\nItem: Magazines and catalogues', metadata={'source': '..\\data\\recyclebc.csv', 'row': 1}),
 Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Paper\nItem: Telephone books and other directories', metadata={'source': '..\\data\\recyclebc.csv', 'row': 2}),
 Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Paper\nItem: Writing and home office paper, paper envelopes, and notepads', metadata={'source': '..\\data\\recyclebc.csv', 'row': 3}),
 Document(page_content='C

In [61]:
retriever_dict = create_retriever(doc_dict[1], 'recycle')
retriever_dict

{'CoV': VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x00000226CA1DC5D0>, search_type='similarity', search_kwargs={}),
 'recycle': VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x00000226FB2BB5D0>, search_type='similarity', search_kwargs={})}

### Create tools

In [63]:
description_dict['recycle'] = """
From the Recycle BC website, this document indicates whether or not an item is accepted for recycling and where to recycle it.
"""
tools_list_dict[2] = create_tools_list(retriever_dict, description_dict)
tools_list_dict[2]

[Tool(name='search_CoV', description='Searches and returns documents regarding waste and recycling in the City of Vancouver.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x00000226CA1DC5D0>, search_type='similarity', search_kwargs={})>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x00000226CA1DC5D0>, search_type='similarity', search_kwargs={})>),
 Tool(name='search_recycle', description='\nFrom the Recycle BC website, this document indicates whether or not an item is accepted for recycling and where to recycle it.\n', args_schema=None, return_direct=False, verbose=False, callbacks=None, callb

In [64]:
conversation_id = 2
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])

In [65]:
answer_dict[input_id]

{'question': 'Where do I recycle coffee cups in Vancouver?',
 'chat_history': [HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False),
  AIMessage(content='For single-family homes and duplexes in Vancouver, you can recycle coffee cups in your blue bin provided by GFL Environmental. You can contact GFL Environmental at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service, including coffee cup recycling.\n\nFor apartments, condos, and townhomes, waste collection services are not provided by the City of Vancouver. Waste Management Canada collects recycling on behalf of Recycle BC for these types of buildings. You can contact Waste Management Canada at 604-282-7961 for questions and issues about your recycling service, including coffee cup recycling.', additional_kwargs={}, example=False)],
 'answer': 'For single-family homes and duplexes in Vancouver, you can recycle coffee cups in your b

#### Update tool descriptions

In [66]:
description_dict['recycle'] = """
From the Recycle BC website, this document provides the most specific information 
about whether or not an item is accepted for recycling and where to recycle it.
This should be the main resource for recycling information for residents of British Columbia.
"""
tools_list_dict[2.1] = create_tools_list(retriever_dict, description_dict)
conversation_id = 2
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot(vector_store=vector_dict['recycle'])
answer_dict[input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])

In [67]:
answer_dict[input_id]

{'question': 'Where do I recycle coffee cups in Vancouver?',
 'chat_history': [HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False),
  AIMessage(content='You can recycle coffee cups in Vancouver through curbside collection, multi-family collection, and at Recycle BC depots. Coffee cups fall under the category of "Cartons and Paper Cups" and are accepted for recycling.', additional_kwargs={}, example=False)],
 'answer': 'You can recycle coffee cups in Vancouver through curbside collection, multi-family collection, and at Recycle BC depots. Coffee cups fall under the category of "Cartons and Paper Cups" and are accepted for recycling.'}

# Iteration 3 allow for dynamic retrieval

In [73]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder


def create_chatbot(tools=tools_list, vector_store=vector_dict['CoV'], verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools, verbose)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer.
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            Here is your chat history with the resident: \n
            """
        )
    )
    instructions = """
    Respond to the resident's query, which are delimited by triple backticks: ```{question}```
    """
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name='chat_history'), instructions]
    )

    # prompt = PromptTemplate(
    #     input_variables=['chat_history', 'question'], 
    #     output_parser=None, partial_variables={}, 
    #     template=template, template_format='f-string', validate_template=True)
    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    return agent_executor

def chat_with_chatbot(user_input, agent_executor, verbose=True):
    result = chat({"question": user_input})
    
    return result

conversation_id = 3
answer_id = 1
answer_dict[conversation_id] = dict()
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]

ValidationError: 3 validation errors for ChatPromptTemplate
messages -> 2
  value is not a valid dict (type=type_error.dict)
messages -> 2
  value is not a valid dict (type=type_error.dict)
messages -> 2
  value is not a valid dict (type=type_error.dict)

## Iteration 3.1

In [78]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder


def create_chatbot(tools=tools_list, vector_store=vector_dict['CoV'], verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools, verbose)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer.
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            Here is your chat history with the resident: \n
            """
        )
    )
    instructions = """
    Respond to the resident's query, which are delimited by triple backticks: ```{question}```
    """
    instruction_prompt = PromptTemplate.from_template(
        input_variables=['question'], 
        output_parser=None, partial_variables={}, 
        template=instructions, template_format='f-string', validate_template=True)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history'), 
            instruction_prompt
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    return agent_executor

def chat_with_chatbot(user_input, agent_executor, verbose=True):
    result = chat({"question": user_input})
    
    return result

conversation_id = 3
answer_id = 1
answer_dict[conversation_id] = dict()
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]

 25%|██▌       | 1/4 [2:55:29<8:46:27, 10529.26s/it]


TypeError: langchain.prompts.prompt.PromptTemplate() got multiple values for keyword argument 'input_variables'

## 3.2

In [92]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder


def create_chatbot(tools=tools_list, vector_store=vector_dict['CoV'], verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools, verbose)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer.
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            Here is your chat history with the resident: \n
            """
        )
    )
    instructions = "Respond to the resident's query, which are delimited by triple backticks: ```{question}```"
    instruction_prompt = PromptTemplate.from_template(
        instructions
        # input_variables=['question'], 
        # output_parser=None, partial_variables={}, 
        # template=instructions, 
        # template_format='f-string', validate_template=True
        )
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history'), 
            instruction_prompt
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    return agent_executor

def chat_with_chatbot(user_input, agent_executor, verbose=True):
    result = chat({"question": user_input})
    
    return result

conversation_id = 3
answer_id = 1
answer_dict[conversation_id] = dict()
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]

ValidationError: 3 validation errors for ChatPromptTemplate
messages -> 2
  Can't instantiate abstract class BaseMessagePromptTemplate with abstract methods format_messages, input_variables (type=type_error)
messages -> 2
  Can't instantiate abstract class BaseMessage with abstract method type (type=type_error)
messages -> 2
  Can't instantiate abstract class BaseChatPromptTemplate with abstract method format_messages (type=type_error)

In [94]:
instructions = "Respond to the resident's query, which are delimited by triple backticks: ```{question}```"
instruction_prompt = PromptTemplate.from_template(
    instructions
    )
instruction_prompt

PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template="Respond to the resident's query, which are delimited by triple backticks: ```{question}```", template_format='f-string', validate_template=True)

In [97]:
instructions = "Respond to the resident's query, which are delimited by triple backticks: ```{question}```"

instruction_prompt = PromptTemplate(
    input_variables=['question'], 
    output_parser=None, partial_variables={}, 
    template=instructions, template_format='f-string', validate_template=True)
instruction_prompt

PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template="Respond to the resident's query, which are delimited by triple backticks: ```{question}```", template_format='f-string', validate_template=True)

In [100]:
from langchain.prompts import BasePromptTemplate
BasePromptTemplate()

TypeError: Can't instantiate abstract class BasePromptTemplate with abstract methods format, format_prompt

In [103]:
from langchain.prompts.chat import BaseMessagePromptTemplate
BaseMessagePromptTemplate()

TypeError: Can't instantiate abstract class BaseMessagePromptTemplate with abstract methods format_messages, input_variables

## 3.3

In [107]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor

def create_chatbot(tools=tools_list, vector_store=vector_dict['CoV'], verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools, verbose)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer.
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            "Respond to the resident's query, which are delimited by triple backticks: ```{question}```
            """
        ),
        input_variables=['question']
    )
    instruction_prompt = PromptTemplate.from_template(instructions)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    return agent_executor

def chat_with_chatbot(user_input, agent_executor, verbose=True):
    result = agent_executor({"input": user_input})
    
    return result

conversation_id = 3
answer_id = 1
answer_dict[conversation_id] = dict()
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]



> Entering new AgentExecutor chain...

Invoking: `search_CoV` with `recycle coffee cups Vancouver`


[Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/single\n\n\n\nfamily\n\n\n\nhomes\n\n\n\nand\n\n\n\nduplexes.aspx\n\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\n\nYour household waste collection schedule is based on your location.\n\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, including:\n\nCollection questions or issues Replacing blue bins and recycling bags', metadata={'source': '..\\data\\Vancouver single family homes and duplexes.txt'}), Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/apartments\n\n\n\nco

ValueError: One output key expected, got dict_keys(['output', 'intermediate_steps'])

## 3.4

In [117]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

def create_chatbot(tools=tools_list, vector_store=vector_dict['CoV'], verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools, verbose)
    memory = AgentTokenBufferMemory(memory_key='chat_history', llm=llm)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer.
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            "Respond to the resident's query, which are delimited by triple backticks: ```{question}```
            """
        ),
        input_variables=['question']
    )
    instruction_prompt = PromptTemplate.from_template(instructions)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    return agent_executor

def chat_with_chatbot(user_input, agent_executor, verbose=True):
    result = agent_executor({"input": user_input})
    
    return result

conversation_id = 3.4
answer_id = 1
answer_dict[conversation_id] = dict()
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]



> Entering new AgentExecutor chain...

Invoking: `search_CoV` with `recycle coffee cups Vancouver`


[Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/single\n\n\n\nfamily\n\n\n\nhomes\n\n\n\nand\n\n\n\nduplexes.aspx\n\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\n\nYour household waste collection schedule is based on your location.\n\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, including:\n\nCollection questions or issues Replacing blue bins and recycling bags', metadata={'source': '..\\data\\Vancouver single family homes and duplexes.txt'}), Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/apartments\n\n\n\nco

{'input': 'Where do I recycle coffee cups in Vancouver?',
 'chat_history': [HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_CoV', 'arguments': '{\n  "__arg1": "recycle coffee cups Vancouver"\n}'}}, example=False),
  FunctionMessage(content="[Document(page_content='City of Vancouver\\n\\nhttps://vancouver.ca/home\\n\\n\\n\\nproperty\\n\\n\\n\\ndevelopment/single\\n\\n\\n\\nfamily\\n\\n\\n\\nhomes\\n\\n\\n\\nand\\n\\n\\n\\nduplexes.aspx\\n\\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\\n\\nYour household waste collection schedule is based on your location.\\n\\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycl

In [120]:
answer_dict[conversation_id][input_id]['chat_history']

[HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_CoV', 'arguments': '{\n  "__arg1": "recycle coffee cups Vancouver"\n}'}}, example=False),
 FunctionMessage(content="[Document(page_content='City of Vancouver\\n\\nhttps://vancouver.ca/home\\n\\n\\n\\nproperty\\n\\n\\n\\ndevelopment/single\\n\\n\\n\\nfamily\\n\\n\\n\\nhomes\\n\\n\\n\\nand\\n\\n\\n\\nduplexes.aspx\\n\\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\\n\\nYour household waste collection schedule is based on your location.\\n\\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, including:\\n\\nCollection questions or issues Replacing blue bi

In [121]:
answer_dict[conversation_id][input_id]['output']

"You can recycle coffee cups in Vancouver through the recycling services provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC. You can contact them at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service, including the recycling of coffee cups.\n\nPlease note that this information is specific to single-family homes and duplexes. If you live in an apartment, condo, or townhome, Waste Management Canada collects recycling on behalf of Recycle BC. You can reach Waste Management Canada at 604-282-7961 for any questions or issues regarding your recycling service.\n\nLet me know if there's anything else I can help with!"

In [122]:
documents

[Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/apartments\n\n\n\ncondos\n\n\n\nand\n\n\n\ntownhomes.aspx\n\nApartments, condos, and townhomes\n\nWe do not provide waste collection services to most multi-unit buildings.\n\nHowever, we do provide information and resources to building owners and managers to manage waste at apartments, condos, and townhomes.\n\nWaste Management Canada collects your recycling on behalf of Recycle BC External website, opens in new tab Phone Waste Management Canada at 604-282-7961 for questions and issues about your recycling service.', metadata={'source': '..\\data\\Vancouver apartments condos townhomes.txt'}),
 Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/single\n\n\n\nfamily\n\n\n\nhomes\n\n\n\nand\n\n\n\nduplexes.aspx\n\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes an

In [ ]:
query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot()
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]

## 3.5

In [123]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

def create_chatbot(tools, verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools, verbose)
    memory = AgentTokenBufferMemory(memory_key='chat_history', llm=llm)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer.
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            "Respond to the resident's query, which are delimited by triple backticks: ```{question}```
            """
        ),
        input_variables=['question']
    )
    instruction_prompt = PromptTemplate.from_template(instructions)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    return agent_executor

def chat_with_chatbot(user_input, agent_executor, verbose=True):
    result = agent_executor({"input": user_input})
    
    return result

conversation_id = 3.4
answer_dict[conversation_id] = dict()
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]



> Entering new AgentExecutor chain...

Invoking: `search_CoV` with `coffee cups`


[Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/single\n\n\n\nfamily\n\n\n\nhomes\n\n\n\nand\n\n\n\nduplexes.aspx\n\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\n\nYour household waste collection schedule is based on your location.\n\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, including:\n\nCollection questions or issues Replacing blue bins and recycling bags', metadata={'source': '..\\data\\Vancouver single family homes and duplexes.txt'}), Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/apartments\n\n\n\ncondos\n\n\n\nand\n\

{'input': 'Where do I recycle coffee cups in Vancouver?',
 'chat_history': [HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_CoV', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False),
  FunctionMessage(content="[Document(page_content='City of Vancouver\\n\\nhttps://vancouver.ca/home\\n\\n\\n\\nproperty\\n\\n\\n\\ndevelopment/single\\n\\n\\n\\nfamily\\n\\n\\n\\nhomes\\n\\n\\n\\nand\\n\\n\\n\\nduplexes.aspx\\n\\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\\n\\nYour household waste collection schedule is based on your location.\\n\\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, inclu

In [124]:
answer_dict[conversation_id][input_id]['chat_history']

[HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_CoV', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False),
 FunctionMessage(content="[Document(page_content='City of Vancouver\\n\\nhttps://vancouver.ca/home\\n\\n\\n\\nproperty\\n\\n\\n\\ndevelopment/single\\n\\n\\n\\nfamily\\n\\n\\n\\nhomes\\n\\n\\n\\nand\\n\\n\\n\\nduplexes.aspx\\n\\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\\n\\nYour household waste collection schedule is based on your location.\\n\\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, including:\\n\\nCollection questions or issues Replacing blue bins and recycling b

In [125]:
answer_dict[conversation_id][input_id]['output']

"To recycle coffee cups in Vancouver, you can follow these guidelines:\n\n1. For single-family homes and duplexes: The City of Vancouver provides waste collection services for residential garbage, food scraps, and yard waste. Recycling services are provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC. You can contact GFL Environmental at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service, including coffee cup recycling.\n\n2. For apartments, condos, and townhomes: The City of Vancouver does not provide waste collection services to most multi-unit buildings. However, Waste Management Canada collects recycling on behalf of Recycle BC for apartments, condos, and townhomes. You can contact Waste Management Canada at 604-282-7961 for any questions or issues regarding your recycling service, including coffee cup recycling.\n\nPlease note that it's important to check with your specific waste management ser

In [126]:
tools_list_dict[2]

[Tool(name='search_CoV', description='Searches and returns documents regarding waste and recycling in the City of Vancouver.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x00000226CA1DC5D0>, search_type='similarity', search_kwargs={})>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x00000226CA1DC5D0>, search_type='similarity', search_kwargs={})>),
 Tool(name='search_recycle', description='\nFrom the Recycle BC website, this document indicates whether or not an item is accepted for recycling and where to recycle it.\n', args_schema=None, return_direct=False, verbose=False, callbacks=None, callb

### specify which document to check

In [127]:
input_id = 2
query = "According to Recycle BC, where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]['output']



> Entering new AgentExecutor chain...

Invoking: `search_recycle` with `coffee cups`


[Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Cartons and Paper Cups\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\data\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\nLocation: \nCategory: Plastic Containers\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\data\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic Containers\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids with recycling', metadata={'source': '..\\data\\recyclebc.csv', 'row': 42}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic 

'According to Recycle BC, coffee cups for hot and cold beverages can be recycled in Vancouver. You can recycle them through curbside collection, multi-family collection, or at Recycle BC depots.'

In [128]:
input_id = 2.1
query = "How about pop cans?"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]['output']



> Entering new AgentExecutor chain...

Invoking: `search_recycle` with `pop cans`


[Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic Containers\nItem: Plastic bottles with screw caps, spray pump, or pull-up tops for food, dish soap, mouthwash, shampoos, conditioners and other personal care products, pills and vitamins, laundry products, etc.', metadata={'source': '..\\data\\recyclebc.csv', 'row': 32}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic Containers\nItem: Plastic jars with wide mouths and screw-top lids for peanut butter, jam, nuts, condiments, vitamins and supplements, personal care products, etc.', metadata={'source': '..\\data\\recyclebc.csv', 'row': 33}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic 

'Pop cans are not specifically mentioned in the recycling guidelines, but aluminum cans in general are accepted for recycling. You can recycle aluminum cans, including pop cans, through curbside collection, multi-family collection, and Recycle BC depots.'

In [131]:
answer_dict[conversation_id][input_id]['chat_history']

[HumanMessage(content='How about pop cans?', additional_kwargs={}, example=False),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n"__arg1": "pop cans"\n}'}}, example=False),
 FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Plastic bottles with screw caps, spray pump, or pull-up tops for food, dish soap, mouthwash, shampoos, conditioners and other personal care products, pills and vitamins, laundry products, etc.', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 32}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Plastic jars with wide mouths and screw-top lids for peanut butter, jam, nuts, condiments, vitamins and supplements, personal care products, etc.', metadata={'source'

In [132]:
answer_dict[conversation_id][input_id]

{'input': 'How about pop cans?',
 'chat_history': [HumanMessage(content='How about pop cans?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n"__arg1": "pop cans"\n}'}}, example=False),
  FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Plastic bottles with screw caps, spray pump, or pull-up tops for food, dish soap, mouthwash, shampoos, conditioners and other personal care products, pills and vitamins, laundry products, etc.', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 32}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Plastic jars with wide mouths and screw-top lids for peanut butter, jam, nuts, condiments, vitamins and supplement

In [137]:
from pprint import pprint

In [140]:
conversation_dict[conversation_id].memory.chat_memory.messages

[HumanMessage(content='How about pop cans?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n"__arg1": "pop cans"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Plastic bottles with screw caps, spray pump, or pull-up tops for food, dish soap, mouthwash, shampoos, conditioners and other personal care products, pills and vitamins, laundry products, etc.', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 32}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Plastic jars with wide mouths and screw-top lids for peanut butter, jam, nuts, condiments, vitamins and supplements, personal care products, etc.', metadata={'source': 

In [141]:


def invoke_chatbot(user_input, agent_executor, verbose=True):
    result = agent_executor({"input": user_input})
    
    return result


answer_dict[conversation_id][input_id] = invoke_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]['output']



> Entering new AgentExecutor chain...

Invoking: `search_CoV` with `pop cans`


[Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/apartments\n\n\n\ncondos\n\n\n\nand\n\n\n\ntownhomes.aspx\n\nApartments, condos, and townhomes\n\nWe do not provide waste collection services to most multi-unit buildings.\n\nHowever, we do provide information and resources to building owners and managers to manage waste at apartments, condos, and townhomes.\n\nWaste Management Canada collects your recycling on behalf of Recycle BC External website, opens in new tab Phone Waste Management Canada at 604-282-7961 for questions and issues about your recycling service.', metadata={'source': '..\\data\\Vancouver apartments condos townhomes.txt'}), Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/single\n\n\n\nfamily\n\n\n\nhomes\n\n\n\nand\n\n\n\nduplexes.aspx\n\nSingle family homes and duplexes We co

'Pop cans can be recycled in the City of Vancouver. For single-family homes and duplexes, recycling services are provided by GFL Environmental on behalf of Recycle BC. You can contact GFL Environmental at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service, including the recycling of pop cans. For apartments, condos, and townhomes, waste collection services are not provided by the City of Vancouver. However, Waste Management Canada collects recycling on behalf of Recycle BC. You can contact Waste Management Canada at 604-282-7961 for any questions or issues about your recycling service.'

In [146]:
print(answer_dict[conversation_id][input_id].keys())
answer_dict[conversation_id][input_id]['chat_history']

dict_keys(['input', 'chat_history', 'output', 'intermediate_steps'])


[HumanMessage(content='How about pop cans?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n"__arg1": "pop cans"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Plastic bottles with screw caps, spray pump, or pull-up tops for food, dish soap, mouthwash, shampoos, conditioners and other personal care products, pills and vitamins, laundry products, etc.', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 32}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Plastic jars with wide mouths and screw-top lids for peanut butter, jam, nuts, condiments, vitamins and supplements, personal care products, etc.', metadata={'source': 

# iteration 4 get the memory

In [147]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

def create_chatbot(tools, verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools, verbose)
    memory = AgentTokenBufferMemory(memory_key='chat_history', llm=llm)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer.
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            Recycle BC is the main resource for recycling information for residents of British Columbia. 
            Other documents provide additional information not found through Recycle BC.
            Respond to the resident's query, which are delimited by triple backticks: ```{question}```
            """
        ),
        input_variables=['question']
    )
    instruction_prompt = PromptTemplate.from_template(instructions)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'memory': memory,
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info, verbose=True):
    result = agent_info['agent_executor']({"input": user_input})
    
    return result

conversation_id = 4
answer_dict[conversation_id] = dict()
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]



> Entering new AgentExecutor chain...

Invoking: `search_CoV` with `coffee cups`


[Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/single\n\n\n\nfamily\n\n\n\nhomes\n\n\n\nand\n\n\n\nduplexes.aspx\n\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\n\nYour household waste collection schedule is based on your location.\n\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, including:\n\nCollection questions or issues Replacing blue bins and recycling bags', metadata={'source': '..\\data\\Vancouver single family homes and duplexes.txt'}), Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/apartments\n\n\n\ncondos\n\n\n\nand\n\

{'input': 'Where do I recycle coffee cups in Vancouver?', 'chat_history': [HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_CoV', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='City of Vancouver\\n\\nhttps://vancouver.ca/home\\n\\n\\n\\nproperty\\n\\n\\n\\ndevelopment/single\\n\\n\\n\\nfamily\\n\\n\\n\\nhomes\\n\\n\\n\\nand\\n\\n\\n\\nduplexes.aspx\\n\\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\\n\\nYour household waste collection schedule is based on your location.\\n\\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, including:

In [149]:
answer_dict[conversation_id][input_id]['output']

"To recycle coffee cups in Vancouver, you can follow these guidelines:\n\n1. For single-family homes and duplexes: Recycling services for these types of residences are provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC. You can contact GFL Environmental at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service, including coffee cup recycling.\n\n2. For apartments, condos, and townhomes: Waste Management Canada collects recycling for these types of buildings on behalf of Recycle BC. You can contact Waste Management Canada at 604-282-7961 for any questions or issues regarding your recycling service, including coffee cup recycling.\n\nPlease note that it's important to check with your specific waste management provider for any specific instructions or requirements for recycling coffee cups."

In [150]:
conversation_dict[conversation_id]['memory'].chat_memory.messages

[HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_CoV', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='City of Vancouver\\n\\nhttps://vancouver.ca/home\\n\\n\\n\\nproperty\\n\\n\\n\\ndevelopment/single\\n\\n\\n\\nfamily\\n\\n\\n\\nhomes\\n\\n\\n\\nand\\n\\n\\n\\nduplexes.aspx\\n\\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\\n\\nYour household waste collection schedule is based on your location.\\n\\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, including:\\n\\nCollection questions or issues Replacing blue bins and recycling bag

In [151]:
conversation_id = 4
answer_dict[conversation_id] = dict()
input_id = 1.1

query = "What does Recycle BC say about that?"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]



> Entering new AgentExecutor chain...
I'm sorry, could you please clarify what "that" refers to?

> Finished chain.


{'input': 'What does Recycle BC say about that?', 'chat_history': [HumanMessage(content='What does Recycle BC say about that?', additional_kwargs={}, example=False), AIMessage(content='I\'m sorry, could you please clarify what "that" refers to?', additional_kwargs={}, example=False)], 'output': 'I\'m sorry, could you please clarify what "that" refers to?', 'intermediate_steps': []}

In [152]:
answer_dict[conversation_id][input_id]['output']

'I\'m sorry, could you please clarify what "that" refers to?'

In [153]:
conversation_id = 4
answer_dict[conversation_id] = dict()
input_id = 1.2

query = "About coffee ups"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]



> Entering new AgentExecutor chain...
Are you asking about coffee cups and their recycling options?

> Finished chain.


{'input': 'About coffee ups', 'chat_history': [HumanMessage(content='About coffee ups', additional_kwargs={}, example=False), AIMessage(content='Are you asking about coffee cups and their recycling options?', additional_kwargs={}, example=False)], 'output': 'Are you asking about coffee cups and their recycling options?', 'intermediate_steps': []}

In [154]:
answer_dict[conversation_id][input_id]['output']

'Are you asking about coffee cups and their recycling options?'

## 4.1 Previous iterations did not save chat history
Memory was only for intermediate steps.

In [157]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

def create_chatbot(tools, verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools, verbose)
    memory = AgentTokenBufferMemory(memory_key='chat_history', llm=llm)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer.
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            Recycle BC is the main resource for recycling information. 
            Respond to the resident's query, which are delimited by triple backticks: ```{question}```
            """
        ),
        input_variables=['question']
    )
    instruction_prompt = PromptTemplate.from_template(instructions)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'memory': memory,
        'chat_history': []
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):

    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": agent_info['chat_history']
        })
    agent_info['chat_history'].append(result['chat_history'])
    
    return result

conversation_id = 4.1
answer_dict[conversation_id] = dict()
input_id = 1

query = "Where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]



> Entering new AgentExecutor chain...

Invoking: `search_CoV` with `coffee cups`


[Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/single\n\n\n\nfamily\n\n\n\nhomes\n\n\n\nand\n\n\n\nduplexes.aspx\n\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\n\nYour household waste collection schedule is based on your location.\n\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, including:\n\nCollection questions or issues Replacing blue bins and recycling bags', metadata={'source': '..\\data\\Vancouver single family homes and duplexes.txt'}), Document(page_content='City of Vancouver\n\nhttps://vancouver.ca/home\n\n\n\nproperty\n\n\n\ndevelopment/apartments\n\n\n\ncondos\n\n\n\nand\n\

{'input': 'Where do I recycle coffee cups in Vancouver?', 'chat_history': [HumanMessage(content='Where do I recycle coffee cups in Vancouver?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_CoV', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='City of Vancouver\\n\\nhttps://vancouver.ca/home\\n\\n\\n\\nproperty\\n\\n\\n\\ndevelopment/single\\n\\n\\n\\nfamily\\n\\n\\n\\nhomes\\n\\n\\n\\nand\\n\\n\\n\\nduplexes.aspx\\n\\nSingle family homes and duplexes We collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\\n\\nYour household waste collection schedule is based on your location.\\n\\nRecycling: service is provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC External website, opens in new tab Phone 604-282-7966 or email srrecycle@gflenv.com about your recycling service, including:

In [158]:
answer_dict[conversation_id][input_id]['output']

"To recycle coffee cups in Vancouver, you can follow these guidelines based on the type of residence:\n\n1. Single family homes and duplexes: Recycling services for these types of residences are provided by GFL Environmental (formerly Smithrite Disposal Ltd) on behalf of Recycle BC. You can contact GFL Environmental at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service, including coffee cup recycling.\n\n2. Apartments, condos, and townhomes: Waste Management Canada collects recycling for most multi-unit buildings on behalf of Recycle BC. You can contact Waste Management Canada at 604-282-7961 for any questions or issues regarding your recycling service, including coffee cup recycling.\n\nPlease note that it's always a good idea to rinse out the coffee cups before recycling them to ensure they are clean."

## 4.2

In [159]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

def create_chatbot(tools, verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    agent_executor = create_conversational_retrieval_agent(llm, tools, verbose)
    memory = AgentTokenBufferMemory(memory_key='chat_history', llm=llm)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer. 
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            Recycle BC is the main resource for recycling information. 
            Respond to the resident's query, which are delimited by triple backticks: ```{question}```
            """
        ),
        input_variables=['question']
    )
    instruction_prompt = PromptTemplate.from_template(instructions)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'memory': memory,
        'chat_history': []
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):

    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": agent_info['chat_history']
        })
    agent_info['chat_history'].append(result['output'])
    
    return result

conversation_id = 4.2
answer_dict[conversation_id] = dict()
input_id = 1

query = "According to Recycle BC, where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]['output']



> Entering new AgentExecutor chain...

Invoking: `search_recycle` with `coffee cups`


[Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Cartons and Paper Cups\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\data\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\nLocation: \nCategory: Plastic Containers\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\data\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic Containers\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids with recycling', metadata={'source': '..\\data\\recyclebc.csv', 'row': 42}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic 

'According to Recycle BC, coffee cups for hot and cold beverages can be recycled in Vancouver. You can recycle them through curbside collection, multi-family collection, or at Recycle BC depots.'

In [171]:
conversation_dict[4.2]['chat_history']

['According to Recycle BC, coffee cups for hot and cold beverages can be recycled in Vancouver. You can recycle them through curbside collection, multi-family collection, or at Recycle BC depots.']

In [160]:
conversation_id = 4.21
answer_dict[conversation_id] = dict()
input_id = 1.1

query = "What if I live in an apartment building"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id][input_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id][input_id]['output']



> Entering new AgentExecutor chain...
If you live in an apartment building, you may have access to recycling facilities within your building. Many apartment buildings in Metro Vancouver provide recycling bins for residents to use. These bins are usually located in a designated area such as a recycling room or a garbage disposal area.

To find out more about recycling in your specific apartment building, you can check with your building management or strata council. They should be able to provide you with information on what items can be recycled and where the recycling bins are located.

If you have any specific questions about recycling in apartment buildings or need information on how to recycle certain items, feel free to ask!

> Finished chain.


'If you live in an apartment building, you may have access to recycling facilities within your building. Many apartment buildings in Metro Vancouver provide recycling bins for residents to use. These bins are usually located in a designated area such as a recycling room or a garbage disposal area.\n\nTo find out more about recycling in your specific apartment building, you can check with your building management or strata council. They should be able to provide you with information on what items can be recycled and where the recycling bins are located.\n\nIf you have any specific questions about recycling in apartment buildings or need information on how to recycle certain items, feel free to ask!'

In [167]:
print(len(conversation_dict[conversation_id]['chat_history']))
conversation_dict[conversation_id]['chat_history']

1


['If you live in an apartment building, you may have access to recycling facilities within your building. Many apartment buildings in Metro Vancouver provide recycling bins for residents to use. These bins are usually located in a designated area such as a recycling room or a garbage disposal area.\n\nTo find out more about recycling in your specific apartment building, you can check with your building management or strata council. They should be able to provide you with information on what items can be recycled and where the recycling bins are located.\n\nIf you have any specific questions about recycling in apartment buildings or need information on how to recycle certain items, feel free to ask!']

In [172]:
conversation_dict[4.21]['chat_history']

['If you live in an apartment building, you may have access to recycling facilities within your building. Many apartment buildings in Metro Vancouver provide recycling bins for residents to use. These bins are usually located in a designated area such as a recycling room or a garbage disposal area.\n\nTo find out more about recycling in your specific apartment building, you can check with your building management or strata council. They should be able to provide you with information on what items can be recycled and where the recycling bins are located.\n\nIf you have any specific questions about recycling in apartment buildings or need information on how to recycle certain items, feel free to ask!']

In [175]:
answer_dict[conversation_id][input_id].keys()

dict_keys(['input', 'chat_history', 'output', 'intermediate_steps'])

## 4.23

In [177]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

def create_chatbot(tools, verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    memory = AgentTokenBufferMemory(memory_key='chat_history', llm=llm)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer. 
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            Recycle BC is the main resource for recycling information. 
            Respond to the resident's query, which are delimited by triple backticks: ```{question}```
            """
        ),
        input_variables=['question']
    )
    instruction_prompt = PromptTemplate.from_template(instructions)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'memory': memory,
        'chat_history': []
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):

    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": agent_info['chat_history']
        })
    agent_info['chat_history'].append(result['chat_history'])
    
    return result

conversation_id = 4.23
answer_dict[conversation_id] = dict()
input_id = 1

query = "According to Recycle BC, where do I recycle coffee cups in Vancouver?"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id]['output']



> Entering new AgentExecutor chain...

Invoking: `search_recycle` with `coffee cups`


[Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Cartons and Paper Cups\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\data\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\nLocation: \nCategory: Plastic Containers\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\data\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic Containers\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids with recycling', metadata={'source': '..\\data\\recyclebc.csv', 'row': 42}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic 

'According to Recycle BC, coffee cups for hot and cold beverages can be recycled in Vancouver. You can recycle them through curbside collection, multi-family collection, or at Recycle BC depots.'

In [179]:
conversation_id = 4.23
answer_dict[conversation_id] = dict()
input_id = 1

query = "I live in an apartment."
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id]['output']



> Entering new AgentExecutor chain...
That's great! How can I assist you today regarding living in an apartment?

> Finished chain.


"That's great! How can I assist you today regarding living in an apartment?"

In [180]:
conversation_id = 4.23
answer_dict[conversation_id] = dict()
input_id = 1

query = "What are my options for recycling coffee cups?"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id] = chat_with_chatbot(query, conversation_dict[conversation_id])
answer_dict[conversation_id]['output']



> Entering new AgentExecutor chain...

Invoking: `search_recycle` with `coffee cups`


[Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Cartons and Paper Cups\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\data\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\nLocation: \nCategory: Plastic Containers\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\data\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic Containers\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids with recycling', metadata={'source': '..\\data\\recyclebc.csv', 'row': 42}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic 

'You have a few options for recycling coffee cups. \n\nPaper cups for hot and cold beverages can be recycled. You can dispose of them in curbside collection, multi-family collection, or Recycle BC depots. \n\nHowever, plastic or foil lids from coffee and tea pods are not accepted for recycling. \n\nIf you have empty single-use coffee and tea pods, you can recycle them by removing the lids and not including the lids with recycling. \n\nPlastic cold drink cups with lids for take-out beverages can also be recycled in curbside collection, multi-family collection, or Recycle BC depots.'

In [182]:
print(len(conversation_dict[conversation_id]['chat_history']))
conversation_dict[conversation_id]['chat_history']

1


[[HumanMessage(content='What are my options for recycling coffee cups?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Cartons and Paper Cups\\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\\nLocation: \\nCategory: Plastic Containers\\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids 

In [183]:
conversation_dict[conversation_id]['memory'].chat_memory.messages

[HumanMessage(content='What are my options for recycling coffee cups?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Cartons and Paper Cups\\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\\nLocation: \\nCategory: Plastic Containers\\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids w

## 4.24

In [184]:
conversation_id = 4.24
answer_dict[conversation_id] = dict()
input_id = 1

query1 = "What are my options for recycling coffee cups?"
query2 = "I live in an apartment."
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id] = chat_with_chatbot(query1, conversation_dict[conversation_id])
answer_dict[conversation_id] = chat_with_chatbot(query2, conversation_dict[conversation_id])
conversation_dict[conversation_id]['chat_history']



> Entering new AgentExecutor chain...

Invoking: `search_recycle` with `coffee cups`


[Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Cartons and Paper Cups\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\data\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\nLocation: \nCategory: Plastic Containers\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\data\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic Containers\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids with recycling', metadata={'source': '..\\data\\recyclebc.csv', 'row': 42}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic 

[[HumanMessage(content='What are my options for recycling coffee cups?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Cartons and Paper Cups\\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\\nLocation: \\nCategory: Plastic Containers\\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids 

In [188]:
print(len(conversation_dict[conversation_id]['chat_history']))
print(conversation_dict[conversation_id]['chat_history'][0])
print(conversation_dict[conversation_id]['chat_history'][1])

2
[HumanMessage(content='What are my options for recycling coffee cups?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Cartons and Paper Cups\\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\\nLocation: \\nCategory: Plastic Containers\\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids

In [189]:
conversation_dict[conversation_id]['chat_history'][0][0]

HumanMessage(content='What are my options for recycling coffee cups?', additional_kwargs={}, example=False)

In [193]:
print(len(conversation_dict[conversation_id]['chat_history'][0]))
conversation_dict[conversation_id]['chat_history'][0][-1]

6


AIMessage(content='If you live in an apartment, you can still recycle coffee cups. Many apartment buildings in Metro Vancouver have multi-family collection programs in place. This means that you can dispose of your coffee cups in the designated recycling bins or containers provided by your building. \n\nIf your apartment building does not have a multi-family collection program, you can take your coffee cups to the nearest Recycle BC depot. Recycle BC depots are located throughout Metro Vancouver and accept a wide range of recyclable materials, including coffee cups. You can find the nearest depot by visiting the Recycle BC website or contacting your local municipality for more information.', additional_kwargs={}, example=False)

In [196]:
len(conversation_dict[conversation_id]['memory'].chat_memory.messages)

6

## 4.3

In [197]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

def create_chatbot(tools, verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    memory = AgentTokenBufferMemory(memory_key='chat_history', llm=llm)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer. 
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            Recycle BC is the main resource for recycling information. 
            Respond to the resident's query, which are delimited by triple backticks: ```{question}```
            """
        ),
        input_variables=['question']
    )
    instruction_prompt = PromptTemplate.from_template(instructions)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'memory': memory,
        'chat_history': []
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):

    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": agent_info['chat_history']
        })
    agent_info['chat_history'].append(result['chat_history'])
    
    return result, agent_info

conversation_id = 4.3
answer_dict[conversation_id] = dict()
input_id = 1

query1 = "What are my options for recycling coffee cups?"
query2 = "I live in an apartment."
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id], conversation_dict[conversation_id] = chat_with_chatbot(query1, conversation_dict[conversation_id])
answer_dict[conversation_id], conversation_dict[conversation_id] = chat_with_chatbot(query2, conversation_dict[conversation_id])
conversation_dict[conversation_id]['chat_history']



> Entering new AgentExecutor chain...

Invoking: `search_recycle` with `coffee cups`


[Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Cartons and Paper Cups\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\data\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\nLocation: \nCategory: Plastic Containers\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\data\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic Containers\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids with recycling', metadata={'source': '..\\data\\recyclebc.csv', 'row': 42}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic 

[[HumanMessage(content='What are my options for recycling coffee cups?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Cartons and Paper Cups\\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\\nLocation: \\nCategory: Plastic Containers\\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids 

In [200]:
print(len(conversation_dict[conversation_id]['chat_history']))
for question in conversation_dict[conversation_id]['chat_history']:
    print(len(question), question)

2
6 [HumanMessage(content='What are my options for recycling coffee cups?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Cartons and Paper Cups\\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\\nLocation: \\nCategory: Plastic Containers\\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Empty single-use coffee and tea pods; remove lids and do not include li

In [201]:
answer_dict[conversation_id]

{'input': 'I live in an apartment.', 'chat_history': [HumanMessage(content='What are my options for recycling coffee cups?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Cartons and Paper Cups\\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\\nLocation: \\nCategory: Plastic Containers\\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Empty single-use coffe

## ** 4.4 Can see chat history here

In [202]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

def create_chatbot(tools, verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    memory = AgentTokenBufferMemory(memory_key='chat_history', llm=llm)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer. 
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            Recycle BC is the main resource for recycling information. 
            Respond to the resident's query, which are delimited by triple backticks: ```{question}```
            """
        ),
        input_variables=['question']
    )
    instruction_prompt = PromptTemplate.from_template(instructions)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'memory': memory,
        'chat_history': []
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):

    print(f'Chat history length: {len(agent_info["chat_history"])}')

    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": agent_info['chat_history']
        })
    agent_info['chat_history'].append(result['chat_history'])
    
    return result, agent_info

conversation_id = 4.3
answer_dict[conversation_id] = dict()
input_id = 1

query1 = "What are my options for recycling coffee cups?"
query2 = "I live in an apartment."
query3 = "Thank you so much!"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id], conversation_dict[conversation_id] = chat_with_chatbot(query1, conversation_dict[conversation_id])
answer_dict[conversation_id], conversation_dict[conversation_id] = chat_with_chatbot(query2, conversation_dict[conversation_id])
for message in conversation_dict[conversation_id]['chat_history']:
    print(len(message), message)

Chat history length: 0


> Entering new AgentExecutor chain...

Invoking: `search_recycle` with `coffee cups`


[Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Cartons and Paper Cups\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\data\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\nLocation: \nCategory: Plastic Containers\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\data\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic Containers\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids with recycling', metadata={'source': '..\\data\\recyclebc.csv', 'row': 42}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC dep

In [203]:
answer_dict[conversation_id], conversation_dict[conversation_id] = chat_with_chatbot(query3, conversation_dict[conversation_id])
for message in conversation_dict[conversation_id]['chat_history']:
    print(len(message), message)

Chat history length: 2


> Entering new AgentExecutor chain...
You're welcome! I'm glad I could help. If you have any more questions, feel free to ask. Happy recycling!

> Finished chain.
8 [HumanMessage(content='What are my options for recycling coffee cups?', additional_kwargs={}, example=False), AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}}, example=False), FunctionMessage(content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Cartons and Paper Cups\\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\\nLocation: \\nCategory: Plastic Containers\\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\

In [212]:
print(len(answer_dict[conversation_id]['chat_history']))
for message in answer_dict[conversation_id]['chat_history']:
    print(message)

8
content='What are my options for recycling coffee cups?' additional_kwargs={} example=False
content='' additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}} example=False
content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Cartons and Paper Cups\\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\\nLocation: \\nCategory: Plastic Containers\\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids with recycling', metadata={'source': '..\\\\dat

In [214]:
for message in conversation_dict[conversation_id]['chat_history'][-1]:
    print(message)

content='What are my options for recycling coffee cups?' additional_kwargs={} example=False
content='' additional_kwargs={'function_call': {'name': 'search_recycle', 'arguments': '{\n  "__arg1": "coffee cups"\n}'}} example=False
content="[Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Cartons and Paper Cups\\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\\nLocation: \\nCategory: Plastic Containers\\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\\\data\\\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\\nCategory: Plastic Containers\\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids with recycling', metadata={'source': '..\\\\data\

## 4.5 Can see chat history here

In [215]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

def create_chatbot(tools, verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        )

    memory = AgentTokenBufferMemory(memory_key='chat_history', llm=llm)
    system_message = SystemMessage(
        content=("""
            You are a helpful assistant who provides concise answers to residents in Metro Vancouver, Canada.
            You ask enough follow up questions as needed to provide the most relevant answer. 
            Where relevant, you retrieve the relevant information from your documents to answer the resident's question.
            Recycle BC is the main resource for recycling information. 
            Respond to the resident's query, which are delimited by triple backticks: ```{question}```
            """
        ),
        input_variables=['question']
    )
    instruction_prompt = PromptTemplate.from_template(instructions)
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, memory=memory, verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'memory': memory,
        'chat_history': []
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):

    print(f'Chat history length: {len(agent_info["chat_history"])}')

    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": agent_info['chat_history']
        })
    agent_info['chat_history'].append(result['chat_history'])
    
    return result

conversation_id = 4.3
answer_dict[conversation_id] = dict()
input_id = 1

query1 = "What are my options for recycling coffee cups?"
query2 = "I live in an apartment."
query3 = "Thank you so much!"
conversation_dict[conversation_id] = create_chatbot(tools_list_dict[2])
answer_dict[conversation_id] = chat_with_chatbot(query1, conversation_dict[conversation_id])
answer_dict[conversation_id] = chat_with_chatbot(query2, conversation_dict[conversation_id])
answer_dict[conversation_id] = chat_with_chatbot(query3, conversation_dict[conversation_id])
for message in conversation_dict[conversation_id]['chat_history'][-1]:
    print(message)

Chat history length: 0


> Entering new AgentExecutor chain...

Invoking: `search_recycle` with `coffee cups`


[Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Cartons and Paper Cups\nItem: Paper cups for hot and cold beverages', metadata={'source': '..\\data\\recyclebc.csv', 'row': 71}), Document(page_content='Conclusion: Not Accepted\nLocation: \nCategory: Plastic Containers\nItem: Plastic or foil lids from coffee and tea pods', metadata={'source': '..\\data\\recyclebc.csv', 'row': 70}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Plastic Containers\nItem: Empty single-use coffee and tea pods; remove lids and do not include lids with recycling', metadata={'source': '..\\data\\recyclebc.csv', 'row': 42}), Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC dep

# *End of Page*